In [73]:
import json
import os
import sys
from copy import deepcopy
from pathlib import Path
import pandas as pd
from utils import load_json
from collections import OrderedDict

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# lib.env.OUTPUT_DIR = lib.env.PROJECT_DIR / 'output'
# lib.env.DATA_DIR = lib.env.PROJECT_DIR / 'data'

In [74]:
ALL_DATASETS = ['1_ALOI', '12_fault', '17_InternetAds', '19_landsat', '20_letter', \
            '22_magic.gamma', '23_mammography', '29_Pima', '30_satellite', '36_speech', \
                '41_Waveform', '47_yeast']
NORMAL_STRATEGIES = ['RandomSampling', 'LeastConfidence', 'MarginSampling', 'EntropySampling', \
        'LeastConfidenceDropout', 'MarginSamplingDropout', 'EntropySamplingDropout', \
        'BALDDropout', 'VarRatio', 'MeanSTD', 'MetaSampling']
EMBEDDING_STRATEGIES = ['KMeansSampling', 'KMeansSamplingGPU', 'KCenterGreedy', 'KCenterGreedyPCA', \
            'BadgeSampling', 'AdversarialBIM', 'AdversarialDeepFool']
WAAL_STRATEGY = 'WAAL'
LPL_STRATEGY = 'LossPredictionLoss'

DETAILS = ['init_labels', 'budget', 'batch_size']
BUDGET_RATIO_RANGE = [0.05, 0.1, 0.25, 0.5, 0.75]

In [75]:
def sort(df, by):
    if isinstance(by, str):
        by = [by]
    return df.sort_values(['n_objects'] + by, ascending=[True] + ['score' not in x for x in by]).reset_index(drop=True)

def load_record(output):
    """
    Load record from stats.json with a single random seed.
    Output: a dictionary with records
    """
    output = Path(output)
    if not output.exists():
        return None
    path = output / 'stats.json'
    if not path.exists():
        print(f'WARNING! This path does not exist: {path}')
        return None
    stats = load_json(path)
    metrics = stats.get('metrics')
    if metrics is None:
        return None

    dataset = stats['dataset']
    budget = stats['budget_ratio']
    batch_size = stats['batch_size']
    init_labels = stats['init_labeled']
    strategy = stats['strategy']
    base_model = stats['model']
    process = OrderedDict(stats['process'])

    r = {
        'dataset': dataset,
        'strategy': strategy,
        'base_model': base_model,
        'init_labels': init_labels,
        'budget': budget,
        'batch_size': batch_size,
        'all_outliers': process.popitem(last=True)[1][1], #TODO: make it more robust
        'F_aucroc': metrics['F-aucroc'],
        'F_aucpr': metrics['F_aucpr'],
        'F_f1': metrics['F_f1'],
    }
    
    return r

def aggregate(df):
    aggrs = dict(
        #TODO: make table that supports ensemble statistics
        # base_model=('base_model', 'first'),
        # budget=('budget', 'first'),
        init_labels=('init_labels', 'first'),
        batch_size=('batch_size', 'first'),
        all_outliers=('all_outliers', 'first'),
        F_aucroc=('F_aucroc', 'mean'),
        F_aucpr=('F_aucpr', 'mean'),
        F_f1=('F_f1', 'first'),
        count=('F_aucroc', 'count'),
    )
    if 'train_score' in df.columns:
        aggrs.update(dict(
            train_score=('train_score', 'mean'),
            train_std=('train_score', 'std'),
        ))
    df = df.groupby(['dataset', 'strategy', 'base_model', 'budget']).agg(**aggrs)
    df['count'] = df['count'].astype(int)
    return df.reset_index().fillna(0.0)

def format_scores(df, precision):
    def f(record):
        for k, v in list(record.items()):
            if isinstance(v, float):
                record[k] = round(v, precision)
        return record
    return df.apply(f, axis=1)

def make_df(outputs_and_names):
    """
    Make dataframe from a list of outputs and names.
    Output: unaggregated dataframe
    """
    df = []
    for output, strategy_name in outputs_and_names:
        record = load_record(output)
        if not record:
            continue
        df.append(record)
    # df = sort(pd.DataFrame(df).fillna(0.0), 'val_score').reset_index(drop=True)
    df = pd.DataFrame(df).fillna(0.0).reset_index(drop=True)
    df.sort_values(['F_aucroc', 'F_aucpr'], ascending=[True, True], inplace=True)
    # df.sort_values('F_aucroc', ascending=True, inplace=True)
    return df

def build_report_all(outputs_and_names):
    df = make_df(outputs_and_names)
    df = aggregate(df)
    df = format_scores(df, 4)
    df = df.set_index(['dataset'] + ['strategy'] + ['base_model'] + DETAILS)
    return df[['all_outliers', 'F_aucroc', 'F_aucpr', 'F_f1', 'count']]

def build_report_single(dataset, base_model, strategy, budget_ratio):
    output = f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}'
    output = Path(output)
    if not output.exists():
        return None
    path = output / 'stats.json'
    if not path.exists():
        print(f'WARNING! This path does not exist: {path}')
        return None
    stats = load_json(path)
    process = stats.get('process')
    if process is None:
        return None
    df = pd.DataFrame.from_dict(process, orient='index')
    df.columns = ['all_X', 'all_outliers', 'new_X','new_outliers' ,'aucroc','aucpr','f1']
    # df = aggregate(df)
    df = format_scores(df, 4)
    return df

#### Comparison of different strategies with the same budget ratio (Base model: Devnet)

In [76]:
budget_ratio = 0.5  # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
BASE_MODEL = 'DevNet'
outputs_and_names = []
for strategy, base_model, datasets in [
    (s, BASE_MODEL, ALL_DATASETS) for s in NORMAL_STRATEGIES + EMBEDDING_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

for strategy, base_model, datasets in [
    (WAAL_STRATEGY, 'WAAL', ALL_DATASETS),
    (LPL_STRATEGY, 'LPL', ALL_DATASETS),
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

build_report_all(outputs_and_names)

all_outliers  \
dataset        strategy               base_model init_labels budget batch_size                 
12_fault       AdversarialBIM         DevNet     136         0.5    68                   331   
               AdversarialDeepFool    DevNet     136         0.5    68                   355   
               BALDDropout            DevNet     136         0.5    68                   373   
               BadgeSampling          DevNet     136         0.5    68                   262   
               EntropySampling        DevNet     136         0.5    68                   356   
               EntropySamplingDropout DevNet     136         0.5    68                   356   
               KCenterGreedy          DevNet     136         0.5    68                   299   
               KCenterGreedyPCA       DevNet     136         0.5    68                   280   
               KMeansSampling         DevNet     136         0.5    68                   269   
               KMeansSamplingGPU      DevNet     136         0.5    68                   256   
               LeastConfidence        DevNet     136         0.5    68                   356   
               LeastConfidenceDropout DevNet     136         0.5    68                   356   
               LossPredictionLoss     LPL        136         0.5    68                   333   
               MarginSampling         DevNet     136         0.5    68                   356   
               MarginSamplingDropout  DevNet     136         0.5    68                   356   
               MeanSTD                DevNet     136         0.5    68                   365   
               MetaSampling           DevNet     136         0.5    68                   403   
               RandomSampling         DevNet     136         0.5    68                   275   
               VarRatio               DevNet     136         0.5    68                   356   
               WAAL                   WAAL       136         0.5    68                   360   
17_InternetAds AdversarialBIM         DevNet     138         0.5    69                   158   
               AdversarialDeepFool    DevNet     138         0.5    69                   160   
               BALDDropout            DevNet     138         0.5    69                   160   
               BadgeSampling          DevNet     138         0.5    69                   198   
               EntropySampling        DevNet     138         0.5    69                   151   
               EntropySamplingDropout DevNet     138         0.5    69                   151   
               KCenterGreedy          DevNet     138         0.5    69                   233   
               KCenterGreedyPCA       DevNet     138         0.5    69                   220   
               KMeansSampling         DevNet     138         0.5    69                   205   
               KMeansSamplingGPU      DevNet     138         0.5    69                   143   
               LeastConfidence        DevNet     138         0.5    69                   151   
               LeastConfidenceDropout DevNet     138         0.5    69                   151   
               MarginSampling         DevNet     138         0.5    69                   151   
               MarginSamplingDropout  DevNet     138         0.5    69                   151   
               MeanSTD                DevNet     138         0.5    69                   166   
               MetaSampling           DevNet     138         0.5    69                   234   
               RandomSampling         DevNet     138         0.5    69                   156   
               VarRatio               DevNet     138         0.5    69                   151   
               WAAL                   WAAL       138         0.5    69                   175   
19_landsat     AdversarialBIM         DevNet     451         0.5    225                  554   
               AdversarialDeepFool    DevNet    

#### Comparison of different strategies with the same budget ratio (Base model: XGBOD)

In [77]:
budget_ratio = 0.5  # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
BASE_MODEL = 'XGBOD'
outputs_and_names = []
for strategy, base_model, datasets in [
    (s, BASE_MODEL, ALL_DATASETS) for s in NORMAL_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

for strategy, base_model, datasets in [
    ('WAAL', 'WAAL', ALL_DATASETS),
    ('LPL', 'LossPredictionLoss', ALL_DATASETS),
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))
build_report_all(outputs_and_names)

all_outliers  \
dataset        strategy               base_model init_labels budget batch_size                 
12_fault       BALDDropout            XGBOD      136         0.5    68                   271   
               EntropySampling        XGBOD      136         0.5    68                   377   
               EntropySamplingDropout XGBOD      136         0.5    68                   377   
               LeastConfidence        XGBOD      136         0.5    68                   377   
               LeastConfidenceDropout XGBOD      136         0.5    68                   377   
               MarginSampling         XGBOD      136         0.5    68                   377   
               MarginSamplingDropout  XGBOD      136         0.5    68                   377   
               MeanSTD                XGBOD      136         0.5    68                   365   
               MetaSampling           XGBOD      136         0.5    68                   336   
               RandomSampling         XGBOD      136         0.5    68                   268   
               VarRatio               XGBOD      136         0.5    68                   377   
               WAAL                   WAAL       136         0.5    68                   360   
17_InternetAds BALDDropout            XGBOD      138         0.5    69                   136   
               EntropySampling        XGBOD      138         0.5    69                   225   
               EntropySamplingDropout XGBOD      138         0.5    69                   225   
               LeastConfidence        XGBOD      138         0.5    69                   225   
               LeastConfidenceDropout XGBOD      138         0.5    69                   225   
               MarginSampling         XGBOD      138         0.5    69                   225   
               MarginSamplingDropout  XGBOD      138         0.5    69                   225   
               MeanSTD                XGBOD      138         0.5    69                   203   
               MetaSampling           XGBOD      138         0.5    69                   218   
               RandomSampling         XGBOD      138         0.5    69                   170   
               VarRatio               XGBOD      138         0.5    69                   225   
               WAAL                   WAAL       138         0.5    69                   175   
19_landsat     BALDDropout            XGBOD      451         0.5    225                  482   
               EntropySampling        XGBOD      451         0.5    225                  851   
               EntropySamplingDropout XGBOD      451         0.5    225                  851   
               LeastConfidence        XGBOD      451         0.5    225                  851   
               LeastConfidenceDropout XGBOD      451         0.5    225                  851   
               MarginSampling         XGBOD      451         0.5    225                  851   
               MarginSamplingDropout  XGBOD      451         0.5    225                  851   
               MeanSTD                XGBOD      451         0.5    225                  796   
               MetaSampling           XGBOD      451         0.5    225                  567   
               RandomSampling         XGBOD      451         0.5    225                  561   
               VarRatio               XGBOD      451         0.5    225                  851   
               WAAL                   WAAL       451         0.5    225                  885   
1_ALOI         BALDDropout            XGBOD      700         0.5    350                  109   
               EntropySampling        XGBOD      700         0.5    350                  172   
               EntropySamplingDropout XGBOD      700         0.5    350                  165   
               LeastConfidence        XGBOD      700         0.5    350                  170   
               LeastConfidenceDropout XGBOD     

#### Comparison of different base models (DevNet vs. XGBOD)

In [78]:
budget_ratio = 0.5  # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
BASE_MODEL_1 = 'DevNet'
BASE_MODEL_2 = 'XGBOD'
outputs_and_names = []
for strategy, base_model, datasets in [
    (s, BASE_MODEL_1, ALL_DATASETS) for s in NORMAL_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

for strategy, base_model, datasets in [
    (s, BASE_MODEL_2, ALL_DATASETS) for s in NORMAL_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

build_report_all(outputs_and_names)

all_outliers  \
dataset        strategy               base_model init_labels budget batch_size                 
12_fault       BALDDropout            DevNet     136         0.5    68                   373   
                                      XGBOD      136         0.5    68                   271   
               EntropySampling        DevNet     136         0.5    68                   356   
                                      XGBOD      136         0.5    68                   377   
               EntropySamplingDropout DevNet     136         0.5    68                   356   
                                      XGBOD      136         0.5    68                   377   
               LeastConfidence        DevNet     136         0.5    68                   356   
                                      XGBOD      136         0.5    68                   377   
               LeastConfidenceDropout DevNet     136         0.5    68                   356   
                                      XGBOD      136         0.5    68                   377   
               MarginSampling         DevNet     136         0.5    68                   356   
                                      XGBOD      136         0.5    68                   377   
               MarginSamplingDropout  DevNet     136         0.5    68                   356   
                                      XGBOD      136         0.5    68                   377   
               MeanSTD                DevNet     136         0.5    68                   365   
                                      XGBOD      136         0.5    68                   365   
               MetaSampling           DevNet     136         0.5    68                   403   
                                      XGBOD      136         0.5    68                   336   
               RandomSampling         DevNet     136         0.5    68                   275   
                                      XGBOD      136         0.5    68                   268   
               VarRatio               DevNet     136         0.5    68                   356   
                                      XGBOD      136         0.5    68                   377   
17_InternetAds BALDDropout            DevNet     138         0.5    69                   160   
                                      XGBOD      138         0.5    69                   136   
               EntropySampling        DevNet     138         0.5    69                   151   
                                      XGBOD      138         0.5    69                   225   
               EntropySamplingDropout DevNet     138         0.5    69                   151   
                                      XGBOD      138         0.5    69                   225   
               LeastConfidence        DevNet     138         0.5    69                   151   
                                      XGBOD      138         0.5    69                   225   
               LeastConfidenceDropout DevNet     138         0.5    69                   151   
                                      XGBOD      138         0.5    69                   225   
               MarginSampling         DevNet     138         0.5    69                   151   
                                      XGBOD      138         0.5    69                   225   
               MarginSamplingDropout  DevNet     138         0.5    69                   151   
                                      XGBOD      138         0.5    69                   225   
               MeanSTD                DevNet     138         0.5    69                   166   
                                      XGBOD      138         0.5    69                   203   
               MetaSampling           DevNet     138         0.5    69                   234   
                                      XGBOD      138         0.5    69                   218   
               RandomSampling         DevNet    

#### Comparison of different budgets

In [79]:
# specify your input here
DATASET = '12_fault'
BASE_MODEL = 'DevNet'
STRATEGY = 'BALDDropout'
outputs_and_names = []

for ratio in BUDGET_RATIO_RANGE:
    outputs_and_names.append((f'output/{DATASET}/{STRATEGY}/{BASE_MODEL}/{ratio}', strategy))

build_report_all(outputs_and_names)

all_outliers  \
dataset  strategy    base_model init_labels budget batch_size                 
12_fault BALDDropout DevNet     136         0.05   32                    71   
                                            0.10   32                   104   
                                            0.25   34                   208   
                                            0.50   68                   373   
                                            0.75   102                  434   

                                                               F_aucroc  \
dataset  strategy    base_model init_labels budget batch_size             
12_fault BALDDropout DevNet     136         0.05   32            0.6785   
                                            0.10   32            0.6765   
                                            0.25   34            0.6838   
                                            0.50   68            0.7294   
                                            0.75   102           0.7056   

                                                               F_aucpr  \
dataset  strategy    base_model init_labels budget batch_size            
12_fault BALDDropout DevNet     136         0.05   32           0.5010   
                                            0.10   32           0.5187   
                                            0.25   34           0.5186   
                                            0.50   68           0.5906   
                                            0.75   102          0.5706   

                                                                          F_f1  \
dataset  strategy    base_model init_labels budget batch_size                    
12_fault BALDDropout DevNet     136         0.05   32          [0.788 ,0.1466]   
                                            0.10   32          [0.7923,0.1638]   
                                            0.25   34          [0.7901,0.1552]   
                                            0.50   68          [0.8009,0.1983]   
                                            0.75   102         [0.7987,0.1897]   

                                                               count  
dataset  strategy    base_model init_labels budget batch_size         
12_fault BALDDropout DevNet     136         0.05   32              1  
                                            0.10   32              1  
                                            0.25   34              1  
                                            0.50   68              1  
                                            0.75   102             1

#### Single active learning process

In [80]:
budget_ratio = 0.5 # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
DATASET = '12_fault'
BASE_MODEL = 'DevNet'
STRATEGY = 'BALDDropout'

build_report_single(DATASET, BASE_MODEL, STRATEGY, budget_ratio)

,all_X,all_outliers,new_X,new_outliers,aucroc,aucpr,f1
round_0,1358,38,68,38,0.6538,0.5086,"[0.7923,0.1638]"
round_1,1358,68,68,30,0.6711,0.5125,"[0.7923,0.1638]"
round_2,1358,95,68,27,0.6970,0.5240,"[0.7923,0.1638]"
round_3,1358,130,68,35,0.7078,0.5600,"[0.7944,0.1724]"
round_4,1358,164,68,34,0.7162,0.5590,"[0.7966,0.181 ]"
round_5,1358,202,68,38,0.7177,0.5661,"[0.7966,0.181 ]"
round_6,1358,248,68,46,0.7218,0.5677,"[0.7966,0.181 ]"
round_7,1358,288,68,40,0.7305,0.5831,"[0.7944,0.1724]"
round_8,1358,306,68,18,0.7266,0.5809,"[0.7966,0.181 ]"
round_9,1358,343,68,37,0.7216,0.5824,"[0.803 ,0.2069]"
